# NLP Lab Session Week 2
# Bigram Frequencies and Mutual Information Scores in NLTK

In [7]:
import nltk
from nltk import FreqDist
import re
from nltk.collocations import *

## Data

Now we want to set up the emma text again for processing by repeating steps that we did last week.  This gets the text of the book Emma, separates it into tokens with the word tokenizer, and converts all the characters to lower case.

In [8]:
print(nltk.corpus.gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [9]:
# get text of the book Emma
file0 = nltk.corpus.gutenberg.fileids()[0]
emmatext = nltk.corpus.gutenberg.raw(file0)
# separate it into tokens
emmatokens = nltk.word_tokenize(emmatext)
# convert to lower case
emmawords = [w.lower() for w in emmatokens]
print("This text has {:d} words. This is a sample of 10 words:".format(len(emmawords)))
print(emmawords[:10])

This text has 191776 words. This is a sample of 10 words:
['[', 'emma', 'by', 'jane', 'austen', '1816', ']', 'volume', 'i', 'chapter']


As before, we create a frequency distribution of the words, using the NLTK FreqDist module/class, and we show the 30 top frequency words.  Since the word frequency items are a pair of (word, frequency), we can use item 0 to get the word and item 1 to get the frequency.  Printing the string ‘\t’ inserts a tab into the output, so that the frequency numbers line up.

In [10]:
# frequency distribution of words from text
ndist = FreqDist(emmawords)
# 10 most common words
nitems = ndist.most_common(10)
for item in nitems:
    print(item[0], '\t', item[1])

, 	 12016
. 	 6346
the 	 5201
to 	 5181
and 	 4877
of 	 4284
i 	 3177
a 	 3124
-- 	 3100
it 	 2503


In [11]:
# another way to get tokenized text
emmawords2 = nltk.corpus.gutenberg.words(file0)
emmawords2lowercase = [w.lower() for w in emmawords2]

Do you think the word lists that emmawords and emmawords2lowercase contain should be identical? Try this out and see if this is what you expected:

In [12]:
print(len(emmawords))
print(len(emmawords2lowercase))

191776
192427


There are more words in emmawords2lowercase because it parses hyphenated words into multiple words.

In [12]:
print(emmawords[:160])

['[', 'emma', 'by', 'jane', 'austen', '1816', ']', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty-one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.', 'she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'s", 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.', 'her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses', ';', 'and', 'her', 'place', 'had', 'been', 'supplied', 'by', 'an'

In [14]:
print(emmawords2lowercase[:160])

['[', 'emma', 'by', 'jane', 'austen', '1816', ']', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty', '-', 'one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.', 'she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'", 's', 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.', 'her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses', ';', 'and', 'her', 'place', 'had', 'been', 'supplied'

## Filtering non-Words

In [13]:
# using regular expressions
# this regular expression pattern matches any word that contains all non-alphabetical
#   lower-case characters [^a-z]+
# the beginning ^ and ending $ require the match to begin and end on a word boundary 
pattern = re.compile('^[^a-z]+$')
nonAlphaMatch = pattern.match('**')

In [14]:
#  if it matched, print a message
if nonAlphaMatch: print ('matched non-alphabetical')

matched non-alphabetical


In [15]:
# function that takes a word and returns true if it consists only of non-alphabetic characters
def alpha_filter(w):
    # pattern to match a word of non-alphabetical characters
    pattern = re.compile('^[^a-z]+$')
    if pattern.match(w):
        return True
    else:
        return False

In [16]:
alphaemmawords = [w for w in emmawords if not alpha_filter(w)] # filter for alphabetic words only
print(len(alphaemmawords))
print(alphaemmawords[:100])

161456
['emma', 'by', 'jane', 'austen', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', 'handsome', 'clever', 'and', 'rich', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', 'and', 'had', 'lived', 'nearly', 'twenty-one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', 'she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', 'indulgent', 'father', 'and', 'had', 'in', 'consequence', 'of', 'her', 'sister', "'s", 'marriage', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', 'her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses']


## Filtering Stop Words

### Stop words should be tailored to the tokenization and the analysis that you're trying to do.

In [17]:
nltkstopwords = nltk.corpus.stopwords.words('english')
print(len(nltkstopwords))
print(nltkstopwords)

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

To accommodate this tokenization, we will add some stopwords that have the apostrophe together with the contraction.  Here is the list that I developed:

In [18]:
morestopwords = ['could','would','might','must','need','sha','wo','y',"'s","'d","'ll","'t","'m","'re","'ve"]
stopwords = nltkstopwords + morestopwords
print(len(stopwords))
print(sorted(stopwords))

194
["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'could', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'might', 'mightn', "mightn't", 'more', 'most', 'must', 'mustn', "mustn't", 'my', 'myself', 'need', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'sha', 'sha

In [19]:
# filtering using stopwords
stoppedemmawords = [w for w in alphaemmawords if not w in stopwords]
print("Before:\n", len(alphaemmawords))
print("After:\n", len(stoppedemmawords))

Before:
 161456
After:
 70579


In [22]:
# make frequency distribution with new filtered word list
emmadist = FreqDist(stoppedemmawords)
emmaitems = emmadist.most_common(30)
print(emmaitems)

[('mr.', 1091), ('emma', 855), ('mrs.', 668), ('miss', 599), ('harriet', 496), ('much', 484), ('said', 483), ('one', 447), ('every', 435), ('weston', 430), ('thing', 392), ('think', 383), ('well', 378), ('elton', 378), ('knightley', 373), ('little', 359), ('never', 358), ('know', 335), ('good', 313), ('say', 310), ('woodhouse', 308), ('jane', 301), ('quite', 282), ('time', 275), ('great', 263), ('nothing', 252), ('dear', 241), ('always', 238), ('man', 232), ('fairfax', 232)]


Note that the stop word list removes a lot of the non content-bearing words in the list, but many functional words remain as well as titles. 

## Bigram Frequency Distributions

In [29]:
# get list of bigrams
emmabigrams = list(nltk.bigrams(emmawords))
print(emmabigrams[:20])

[('[', 'emma'), ('emma', 'by'), ('by', 'jane'), ('jane', 'austen'), ('austen', '1816'), ('1816', ']'), (']', 'volume'), ('volume', 'i'), ('i', 'chapter'), ('chapter', 'i'), ('i', 'emma'), ('emma', 'woodhouse'), ('woodhouse', ','), (',', 'handsome'), ('handsome', ','), (',', 'clever'), ('clever', ','), (',', 'and'), ('and', 'rich'), ('rich', ',')]


But we will obtain a lot more functionality by using the functions from the nltk.collocations package.  One of the places to obtain information about this package is in this section of the NLTK documentation called the HOWTOS.  We note that there are also Trigram functions in addition to the Bigram functions shown here.

[http://www.nltk.org/howto/collocations.html](http://www.nltk.org/howto/collocations.html)

To start using bigrams, we import the collocation finder module. 
from nltk.collocations import *

Collocations are expressions of multiple words which commonly co-occur.

Note that you must use the entire list of emmawords before any filtering or the raw bigrams may not be adjacent and will not be correct.  Start with all the words and then run the filters in the bigram finder.

In [30]:
# variable for the bigram measures
bigram_measures = nltk.collocations.BigramAssocMeasures()
# finder that allows us to call other functions to filter the bigrams collected and give scores to bigrams
finder = BigramCollocationFinder.from_words(emmawords)
# score bigrams by frequency; result is a list consisting of pairs, where each pair is a bigram and its score. The raw_freq measure returns frequency as the ratio of the count of the bigram over the count of total bigrams
scored = finder.score_ngrams(bigram_measures.raw_freq)
print(type(scored))
print("The list is {:d} long".format(len(scored)))
first = scored[0]
print(type(first), first)

<class 'list'>
The list is 63242 long
<class 'tuple'> ((',', 'and'), 0.009813532454530285)


In [31]:
# scores are sorted in order of decreasing frequency
# The scores are the frequencies of the bigrams, normalized to fractions by the total number of bigrams.
for bscore in scored[:10]:
    print(bscore)

((',', 'and'), 0.009813532454530285)
(('.', "''"), 0.00603829467712331)
(("''", '``'), 0.004995411313198732)
((';', 'and'), 0.004520899382613049)
(('to', 'be'), 0.0031547221758718505)
((',', "''"), 0.0030452194226597696)
(('.', 'i'), 0.0029722175871850494)
((',', 'i'), 0.0029670031703654264)
(('of', 'the'), 0.0029148590021691972)
(('in', 'the'), 0.0023204154847321877)


For any finder, we can also apply various filter functions. 

First let’s apply our alpha_filter that we created earlier.  It uses a filter that is applied to the individual words.  Note that the function apply_word_filter changes the bigram collocation in the variable “finder”, and any function which takes a word parameter and returns True or False as a result can be used.

In [32]:
help(alpha_filter)

Help on function alpha_filter in module __main__:

alpha_filter(w)
    # function that takes a word and returns true if it consists only of non-alphabetic characters



In [34]:
# filter out non-alphabetic words
finder.apply_word_filter(alpha_filter)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:10]:
    print(bscore)

(('to', 'be'), 0.0031547221758718505)
(('of', 'the'), 0.0029148590021691972)
(('in', 'the'), 0.0023204154847321877)
(('it', 'was'), 0.0023047722342733187)
(('i', 'am'), 0.00205448022693142)
(('she', 'had'), 0.0017311863841148005)
(('she', 'was'), 0.001710328716836309)
(('had', 'been'), 0.0016008259636242283)
(('it', 'is'), 0.0015538962122476222)
(('i', 'have'), 0.0014652511263140331)


Next we can filter out the stopwords if we wish.  Note that the lambda operates like a function definition “on-the-fly”, i.e. without a function name.

In [35]:
# filter out stopwords
finder.apply_word_filter(lambda w: w in stopwords)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:10]:
    print(bscore)

(('mr.', 'knightley'), 0.00142353579175705)
(('mrs.', 'weston'), 0.0012827465376272318)
(('mr.', 'elton'), 0.0011002419489404304)
(('miss', 'woodhouse'), 0.0008864508593358918)
(('mr.', 'weston'), 0.0008238778575004171)
(('frank', 'churchill'), 0.0007508760220256966)
(('mrs.', 'elton'), 0.0007300183547472051)
(('mr.', 'woodhouse'), 0.000683088603370599)
(('every', 'thing'), 0.0006465876856332388)
(('miss', 'fairfax'), 0.000636158851993993)


## Other filter methods (frequency threshold, word length)

These are "less meaningful" filters.

In [36]:
# filter for words that occur with minimum frequency of 2
finder2 = BigramCollocationFinder.from_words(emmawords)
finder2.apply_freq_filter(2)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:10]:
    print(bscore)

((',', 'and'), 0.009813532454530285)
(('.', "''"), 0.00603829467712331)
(("''", '``'), 0.004995411313198732)
((';', 'and'), 0.004520899382613049)
(('to', 'be'), 0.0031547221758718505)
((',', "''"), 0.0030452194226597696)
(('.', 'i'), 0.0029722175871850494)
((',', 'i'), 0.0029670031703654264)
(('of', 'the'), 0.0029148590021691972)
(('in', 'the'), 0.0023204154847321877)


In [37]:
# filter out bigrams where first word's length is less than 2
finder2.apply_ngram_filter(lambda w1, w2: len(w1) < 2)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:20]:
    print(bscore)

(("''", '``'), 0.004995411313198732)
(('to', 'be'), 0.0031547221758718505)
(('of', 'the'), 0.0029148590021691972)
(('in', 'the'), 0.0023204154847321877)
(('it', 'was'), 0.0023047722342733187)
(('--', 'and'), 0.0017416152177540463)
(('she', 'had'), 0.0017311863841148005)
(('she', 'was'), 0.001710328716836309)
(('had', 'been'), 0.0016008259636242283)
(('it', 'is'), 0.0015538962122476222)
(('could', 'not'), 0.0014496078758551643)
(('mr.', 'knightley'), 0.00142353579175705)
(("''", 'said'), 0.0013818204572000668)
(('``', 'i'), 0.0013609627899215753)
(('of', 'her'), 0.0013557483731019523)
(('--', 'i'), 0.0013401051226430837)
(('mrs.', 'weston'), 0.0012827465376272318)
(('have', 'been'), 0.0012566744535291172)
(('he', 'had'), 0.0012514600367094944)
(('to', 'the'), 0.0012358167862506256)


## Mutual Information and other scorers

Recall that Mutual Information is a score introduced in the paper by Church and Hanks, where they defined it as an Association Ratio. Note that technically the original information theoretic definition of mutual information allows the two words to be in either order, but that the association ratio defined by Church and Hanks requires the words to be in order from left to right wherever they appear in the window

In NLTK, the mutual information score is given by a function for Pointwise Mutual Information, where this is the version without the window.

* Mutual information computes the probability of two words occurring in sequence.
* The more strongly connected two items are, the higher their PMI score.
* But when you apply the Mutual Information score to small documents or documents with rare words, the results don’t really make sense.  
* It is recommended to run the PMI scorer with a minimum frequency of 5, which will make more sense on very large documents.

In [38]:
# no filtering, just scoring based on PMI (pointwise mutual information)
# in order to use PMI, filter out words that have less than the minimum frequency of 5 (per Church and Hanks paper)
finder3 = BigramCollocationFinder.from_words(emmawords)
# filter on words with minimum frequency of 5
finder3.apply_freq_filter(5)
scored = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored[:20]:
    print(bscore)

(('d', "'ye"), 14.964100157849273)
(('sore', 'throat'), 14.089631039933131)
(('brunswick', 'square'), 13.952127516183198)
(('william', 'larkins'), 13.089631039933133)
(('baked', 'apples'), 12.964100157849273)
(('box', 'hill'), 12.735994085318433)
(('sixteen', 'miles'), 12.613602910765142)
(('maple', 'grove'), 12.594866348183555)
(('hair', 'cut'), 12.06363583140019)
(('south', 'end'), 11.964100157849275)
(('colonel', 'campbell'), 11.412166457515587)
(('protest', 'against'), 11.34742879740078)
(('robert', 'martin'), 11.093868032819602)
(('five', 'couple'), 10.841703526489548)
(('vast', 'deal'), 10.762466296679625)
(('ready', 'wit'), 10.652225727625833)
(('donwell', 'abbey'), 10.51931531517638)
(('musical', 'society'), 10.509046979722552)
(('infinitely', 'superior'), 10.230745817235448)
(('married', 'women'), 10.057209562240756)
